In [2]:
from agents import Agent, Runner, trace, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.mcp import MCPServerStdio

from dotenv import load_dotenv
import os

from IPython.display import display, Markdown
from datetime import datetime

load_dotenv(override=True)

True

# 3 MCP Servers
To give these powers:
- Memory
- Internet Search
- Live Market Data

## Server 1 - Knowledge-Graph Based Memory

In [6]:
os.makedirs('./memory', exist_ok=True)

params = {
    "command": "npx",
    "args": [
        "-y",
        "mcp-memory-libsql",
    ],
    "env": {
        "LIBSQL_URL": "file:./memory/ed.db",
    }
}

async with MCPServerStdio(params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

In [7]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversation"
request = "My name is Ed. I'm running a workshop live about AI Agents, and co-presenting with Jon, from the SuperDataScience podcast."

client = AsyncOpenAI(api_key=os.environ['GEMINI_API_KEY'], base_url=os.environ['GEMINI_BASE_URL'])
model = OpenAIChatCompletionsModel('gemini-2.5-flash', client)

In [8]:
async with MCPServerStdio(params) as server:
    agent = Agent(name='agent', instructions=instructions, model=model, mcp_servers=[server])
    
    with trace('conversation'):
        result = await Runner.run(agent, request)
    
    display(Markdown(result.final_output))

That sounds like a very interesting workshop, Ed! Good luck with it. I've stored this information so I can remember it for our future interactions.

In [9]:
question = 'My name is Ed. What do you know about me?'

async with MCPServerStdio(params) as server:
    agent = Agent(name='agent', instructions=instructions, model=model, mcp_servers=[server])
    
    with trace('conversation'):
        result = await Runner.run(agent, question)
    
    display(Markdown(result.final_output))

You are running a workshop about AI Agents and co-presenting it with Jon.